In [ ]:
import os

package_path = '/home/meganorm-smkia/Code/MEGaNorm/'
os.chdir(package_path)

from utils.parallel import submit_jobs, check_jobs_status, collect_results
from datasets.camcan import load_camcan_data
from utils.nm import hbr_data_split

from utils.IO import mergeDataframes
from src.summarizeFeatures import summarizeFeatures

import pcntoolkit as ptk

In [ ]:
### Paths and configs

username = 'meganorm-smkia'

data_path = '/project/meganorm/Data/camcan/CamCAN/cc700/meg/pipeline/release005/BIDSsep/derivatives_rest/aa/AA_movecomp_transdef/aamod_meg_maxfilt_00003/'
target_dir = '/home/meganorm-smkia/Data/CamCAN/Features/'

log_path = '/home/meganorm-smkia/temp/log/'
temp_path = '/home/meganorm-smkia/temp/tmp/'

batch_file = '/home/meganorm-smkia/Code/MEGaNorm/src/batch_job.sh'

if not os.path.isdir(log_path):
    os.makedirs(log_path)

In [ ]:
### Parallel feature extraction

subjects = os.listdir(data_path)
 
# Running Jobs
start_time = submit_jobs(batch_file, data_path, subjects, temp_path)
# Checking jobs
failed_job_names= check_jobs_status(username, start_time)

while len(failed_job_names)>0:
    # Re-running Jobs
    start_time = submit_jobs(batch_file, data_path, failed_job_names, temp_path)
    # Checking jobs
    failed_job_names= check_jobs_status(username, start_time)

collect_results(target_dir, subjects, temp_path, file_name='camcan_features')

In [ ]:
# merge feature dataframes
dergedDf = mergeDataframes(temp_path)
# average features across the whole brain (as a summrization)
summarizeFeatures(dergedDf)
summarizeFeatures.to_csv(?) # TODO

In [ ]:
### Data preparation for Normative Modeling

processing_dir = '/home/meganorm-smkia/temp/tmp/'
camcan_cov_path = '/project/meganorm/Data/camcan/CamCAN/cc700/participants.tsv'
camcan_feature_path = '/home/meganorm-smkia/Data/CamCAN/Features/camcan_features.csv'

camcan_data = load_camcan_data(camcan_feature_path, camcan_cov_path)

hbr_data_split(camcan_data, processing_dir, drop_nans=True, batch_effects=['gender','site'])


In [ ]:
### Setting up NM configs

python_path = '/project/meganorm/Software/Miniconda3/envs/mne/bin/python' 

hbr_configs = {'hetero_SHASH_bspline':{'model_type':'bspline', 'likelihood':'SHASHb', 'linear_sigma':'True',
                                   'random_slope_mu':'True', 'linear_epsilon':'True', 'linear_delta':'True'},
            }

inscaler='minmax' 
outscaler='minmax' 
batch_size = 1
outputsuffix = '_estimate'

respfile = processing_dir + 'y_train.pkl'
covfile = processing_dir + 'x_train.pkl'

testrespfile_path = processing_dir + 'y_test.pkl'
testcovfile_path = processing_dir + 'x_test.pkl'

trbefile = processing_dir + 'b_train.pkl'
tsbefile = processing_dir + 'b_test.pkl'

memory = '2gb'
duration = '10:00:00'
method = 'hetero_SHASH_bspline'
cluster_spec = 'slurm'

In [ ]:
### Running NM

ptk.normative_parallel.execute_nm(processing_dir, python_path,
               'Estimate_', covfile, respfile, batch_size, memory, duration, alg='hbr', 
               log_path=log_path, binary=True, testcovfile_path=testcovfile_path, 
               testrespfile_path=testrespfile_path,trbefile=trbefile, tsbefile=tsbefile, 
               model_type=hbr_configs[method]['model_type'], likelihood=hbr_configs[method]['likelihood'],  
               linear_sigma=hbr_configs[method]['linear_sigma'], random_slope_mu=hbr_configs[method]['random_slope_mu'],
               linear_epsilon=hbr_configs[method]['linear_epsilon'], linear_delta=hbr_configs[method]['linear_delta'], 
               savemodel='True', inscaler=inscaler, outscaler=outscaler, outputsuffix=outputsuffix, 
               interactive='auto', cluster_spec=cluster_spec)